In [1]:
# WHEN CREATING DATA, NEED TO SHIFT INPUT TOKEN INDEXES BY ONE!!!
# 1. Try with real data
# 2. Figure out if this is working the way we expect it to work or not
# 3. Check out all TODO :
# WEIGHT SHARING!!!!!!

In [2]:
# Not implemented in Autograd
# x .+= something -> does not work with Autograd
# x = x .+ something -> works

# Not implemented in Knet
# getindex for 3d KnetArray
# Also there is a bug with std function. Inside it uses _var, which uses iterate, which fails for KnetArrays

In [3]:
# The things I've solved
# 1. bmm with 4 dims
# 2. segment_ids paddings are 0, we can't index with them
# 3. ignoring indexes with labels -1 while calculating loss
# 4. 2d matrix, 3d tensor multiplication

In [4]:
import Base: length, iterate
using Random

mutable struct PreTrainingData
    input_ids
    input_mask
    segment_ids
    mlm_labels
    nsp_labels
    batchsize
    ninstances
    shuffled
end


function PreTrainingData(input_file; batchsize=16, shuffled=true, seq_len=256)
    input_ids = []
    input_mask = []
    segment_ids = []
    mlm_labels = []
    nsp_labels = []
    f = open(input_file)
    tmp = split.(readlines(f), "\t")
    for i in 1:100#length(tmp)
        instance = eval.(Meta.parse.(tmp[i]))
#         push!(input_ids, instance[1] .+ 1)
#         push!(input_mask, instance[2])
#         push!(segment_ids, instance[3] .+ 1)
#         push!(mlm_labels, map(x -> x != -1 ? x + 1 : x, instance[4]))
#         push!(nsp_labels, instance[5] + 1)
        push!(input_ids, (instance[1] .+ 1)[1:seq_len])
        push!(input_mask, instance[2][1:seq_len])
        push!(segment_ids, (instance[3] .+ 1)[1:seq_len])
        push!(mlm_labels, map(x -> x != -1 ? x + 1 : x, instance[4])[1:seq_len])
        push!(nsp_labels, (instance[5] + 1))
    end
    ninstances = length(input_ids)
    return PreTrainingData(input_ids, input_mask, segment_ids, mlm_labels, nsp_labels, batchsize, ninstances, shuffled)
end


function length(d::PreTrainingData)
    d, r = divrem(d.ninstances, d.batchsize)
    return r == 0 ? d : d+1
end

function iterate(d::PreTrainingData, state=ifelse(d.shuffled, randperm(d.ninstances), 1:d.ninstances))

    state === nothing && return nothing

    if length(state) > d.batchsize
        new_state = state[d.batchsize+1:end]
        input_ids = hcat(d.input_ids[state[1:d.batchsize]]...)
        input_mask = hcat(d.input_mask[state[1:d.batchsize]]...)
        segment_ids = hcat(d.segment_ids[state[1:d.batchsize]]...)
        mlm_labels = hcat(d.mlm_labels[state[1:d.batchsize]]...)
        nsp_labels = hcat(d.nsp_labels[state[1:d.batchsize]]...)
    else
        new_state = nothing
        input_ids = hcat(d.input_ids[state]...)
        input_mask = hcat(d.input_mask[state]...)
        segment_ids = hcat(d.segment_ids[state]...)
        mlm_labels = hcat(d.mlm_labels[state]...)
        nsp_labels = hcat(d.nsp_labels[state]...)
    end
    
    return ((input_ids, input_mask, segment_ids, mlm_labels, nsp_labels), new_state)
end

iterate (generic function with 201 methods)

In [5]:
include("model.jl")

load_from_torch (generic function with 1 method)

In [6]:
using SpecialFunctions
gelu(x) = Float32(x .* 0.5 .* (1.0 .+ erf.(x ./ sqrt(2.0))))
gelu(x::Knet.Bcasted) = x .* 0.5 .* (1.0 .+ erf.(x ./ sqrt(2.0)))

gelu (generic function with 2 methods)

In [7]:
mutable struct Config
    embed_size::Int
    vocab_size::Int
    ff_hidden_size::Int
    max_seq_len::Int
    seq_len::Int
    num_segment::Int
    num_heads::Int
    num_encoder::Int
    batchsize::Int
    atype
    pdrop
    attention_pdrop
    func
end

config = Config(768, 30522, 3072, 512, 64, 2, 12, 12, 8, KnetArray{Float32}, 0.1, 0.1, relu)

Config(768, 30522, 3072, 512, 64, 2, 12, 12, 8, KnetArray{Float32,N} where N, 0.1, 0.1, Knet.relu)

In [8]:
input_file = "/scratch/users/omutlu/hyperpartisan/data_20181122_big/pre_test.tsv" # Each line has 5 columns: input ids, input mask, segment ids, masked lm labels, next sentence label
dtrn = PreTrainingData(input_file, batchsize=config.batchsize, seq_len=config.seq_len)

PreTrainingData(Any[[102, 1001, 7571, 1013, 104, 1013, 10232, 1025, 10769, 2100  …  4809, 2000, 1997, 104, 1013, 14767, 104, 2668, 2001, 3306], [102, 1001, 2097, 2117, 4842, 104, 2043, 24666, 17687, 2006  …  2041, 2025, 6168, 1012, 9023, 104, 2032, 2054, 3755, 2001], [102, 1997, 104, 25043, 8820, 2863, 1013, 2152, 2232, 3037  …  2578, 12839, 2057, 1997, 104, 6987, 1012, 12483, 1011, 2153], [102, 1525, 12839, 2212, 1013, 1524, 2044, 2058, 2032, 4481  …  1525, 1997, 3680, 5654, 1025, 3952, 2578, 12839, 1011, 1997], [102, 4120, 1997, 24530, 2051, 1011, 3392, 2044, 2112, 5255  …  1997, 2467, 1025, 1765, 103, 2909, 2206, 2522, 1030, 2058], [102, 1001, 2688, 1012, 5642, 1012, 2261, 104, 1025, 4003  …  1013, 103, 9023, 1999, 104, 2054, 3755, 104, 2682, 104], [102, 1056, 1013, 4002, 2680, 104, 2000, 1038, 4358, 2009  …  2015, 2768, 1999, 3181, 9699, 1011, 2625, 3800, 4388, 7913], [102, 2917, 2007, 1997, 2796, 1011, 2022, 1046, 2229, 2010  …  5276, 29902, 104, 9081, 2164, 2019, 2001, 3614, 8148

In [9]:
model = BertPreTraining(config)

BertPreTraining(Bert(EmbedLayer(Embedding(P(KnetArray{Float32,2}(768,30522))), Embedding(P(KnetArray{Float32,2}(768,512))), Embedding(P(KnetArray{Float32,2}(768,2))), LayerNormalization(P(KnetArray{Float32,1}(768)), P(KnetArray{Float32,1}(768)), 1.0e-12), 64, 0.1), Encoder[Encoder(SelfAttention(Linear3D(P(KnetArray{Float32,2}(768,768)), P(KnetArray{Float32,1}(768))), Linear3D(P(KnetArray{Float32,2}(768,768)), P(KnetArray{Float32,1}(768))), Linear3D(P(KnetArray{Float32,2}(768,768)), P(KnetArray{Float32,1}(768))), Linear3D(P(KnetArray{Float32,2}(768,768)), P(KnetArray{Float32,1}(768))), 12, 64, 768, 64, 8, 0.1, 0.1), LayerNormalization(P(KnetArray{Float32,1}(768)), P(KnetArray{Float32,1}(768)), 1.0e-12), FeedForward(Dense(Linear3D(P(KnetArray{Float32,2}(3072,768)), P(KnetArray{Float32,1}(3072))), 0.0, relu), Linear3D(P(KnetArray{Float32,2}(768,3072)), P(KnetArray{Float32,1}(768))), 0.1), LayerNormalization(P(KnetArray{Float32,1}(768)), P(KnetArray{Float32,1}(768)), 1.0e-12)), Encoder(Sel

In [10]:
(x, attention_mask, segment_ids, mlm_labels, nsp_labels) = first(dtrn)
lss = @diff model(x, segment_ids, mlm_labels, nsp_labels, attention_mask=attention_mask)

T(10.935729)

In [132]:
using PyCall
@pyimport torch
torch_model = torch.load("/scratch/users/omutlu/hyperpartisan/data_20181122_big/pytorch_model.bin")

Dict{Any,Any} with 207 entries:
  "bert.encoder.layer.2.at… => PyObject tensor([[ 0.0641,  0.0921,  0.0645,  ..…
  "bert.encoder.layer.6.ou… => PyObject tensor([0.8757, 0.8678, 0.8159, 0.8294,…
  "bert.encoder.layer.3.ou… => PyObject tensor([0.8370, 0.8401, 0.8245, 0.8292,…
  "bert.encoder.layer.5.at… => PyObject tensor([[ 0.0022,  0.0240, -0.0346,  ..…
  "bert.encoder.layer.0.at… => PyObject tensor([[ 0.0209, -0.0050, -0.0087,  ..…
  "bert.encoder.layer.0.at… => PyObject tensor([[-0.0125,  0.0259, -0.0247,  ..…
  "bert.encoder.layer.4.in… => PyObject tensor([-0.0937, -0.1178, -0.1182,  ...…
  "bert.encoder.layer.4.at… => PyObject tensor([-5.8893e-03, -2.3500e-03,  2.81…
  "bert.encoder.layer.3.at… => PyObject tensor([0.9213, 0.8878, 0.8387, 0.8576,…
  "bert.encoder.layer.9.at… => PyObject tensor([0.8576, 0.8295, 0.7983, 0.7731,…
  "bert.encoder.layer.11.a… => PyObject tensor([[-0.0619, -0.0012, -0.0201,  ..…
  "bert.encoder.layer.10.a… => PyObject tensor([ 0.0225,  0.0150,  0.0542,  0

In [207]:
model = load_from_torch(model, config.num_encoder, config.atype, torch_model)

BertPreTraining(Bert(EmbedLayer(Embedding(KFloat32[0.00618323 -0.0169476 … -0.0499978 -0.0177459; -0.0568268 -0.0541772 … -0.0468287 -0.0793887; … ; -0.0306487 -0.0289911 … -0.0284732 -0.0283612; -0.0130648 -0.0109822 … -0.0201913 0.0571289]), Embedding(KFloat32[0.0223904 -0.00928706 … 0.01999 0.00243514; -0.00443635 -0.0165968 … -0.00554573 -0.021461; … ; -0.00410777 0.0431863 … -0.0115938 0.0253736; 0.0127998 -0.019281 … -0.0258449 -0.0716339]), Embedding(KFloat32[0.00328108 -0.00180102; 0.00675674 0.00141564; … ; -0.00558636 -0.00243301; -0.00825712 -0.00647022]), LayerNormalization(KFloat32[0.921038, 0.892587, 0.840731, 0.872586, 0.910665, 0.903145, 0.928933, 0.91257, 0.954167, 0.805993  …  0.868524, 0.781676, 0.918081, 0.889764, 0.861143, 0.908655, 0.903872, 0.783843, 0.878898, 0.962423], KFloat32[-0.0262504, -0.0262299, 0.0461384, 0.097543, -0.0358597, -0.0405648, 0.0265303, -0.032672, -0.00250848, -0.00376075  …  -0.0107369, -0.0639957, -0.0192386, -0.0656198, -0.0321019, 0.0077

In [208]:
model(dtrn)

Any[11.037]
Any[11.037, 11.3105]
Any[11.037, 11.3105, 11.9327]
Any[11.037, 11.3105, 11.9327, 11.2431]
Any[11.037, 11.3105, 11.9327, 11.2431, 11.2654]


InterruptException: InterruptException:

In [224]:
Knet.log(30522) + Knet.log(2)

11.019350194610766

In [229]:
# Check if there is weight sharing
torch_model["cls.predictions.decoder.weight"][:cpu]()[:numpy]() == torch_model["bert.embeddings.word_embeddings.weight"][:cpu]()[:numpy]()

true

In [ ]:
# Unimportant stuff after this part

In [3]:
#=
asd = randn(2,3,4,5)
dsa = randn(3,2,4,5)
@time begin
    qwe = zeros(2,2,4,5)
    for i in 1:5
        qwe[:, :, :, i] = bmm(asd[:, :, :, i], dsa[:, :, :, i])
    end
end
=#

  1.480823 seconds (3.64 M allocations: 163.294 MiB, 4.27% gc time)


In [4]:
# @time qwe2 = reshape(bmm(reshape(asd, (2,3,20)), reshape(dsa, (3,2,20))), (2,2,4,5))

  0.064232 seconds (184.66 k allocations: 9.384 MiB)


2×2×4×5 Array{Float64,4}:
[:, :, 1, 1] =
  0.115702   0.84845
 -1.28604   -1.00778

[:, :, 2, 1] =
 -0.117182   0.363062
  4.44089   -0.717041

[:, :, 3, 1] =
 -0.733266  -1.53797
 -1.17601   -1.41673

[:, :, 4, 1] =
 0.791245   -0.0872022
 0.0146814   0.0400762

[:, :, 1, 2] =
 -1.13423  0.766976
  2.13047  0.910996

[:, :, 2, 2] =
 1.41911    2.82969 
 0.716323  -0.676642

[:, :, 3, 2] =
  1.1102      -1.53823
 -0.00522836   1.55034

[:, :, 4, 2] =
 -3.75843   0.621332
  1.44112  -1.03046 

[:, :, 1, 3] =
 -0.760699  1.55247
  0.272704  3.22196

[:, :, 2, 3] =
 0.12309  2.14021
 1.64758  2.65152

[:, :, 3, 3] =
 -0.815294   1.14375 
 -0.615639  -0.989944

[:, :, 4, 3] =
 -1.07816     -1.03071 
 -0.00645643   0.314534

[:, :, 1, 4] =
 -0.366951  -0.496234
  2.55274    0.690053

[:, :, 2, 4] =
 -0.176301  -0.385411
 -0.480657   1.25897 

[:, :, 3, 4] =
  2.53924   2.71935
 -2.18906  -1.85169

[:, :, 4, 4] =
 -0.585549  -0.480815
  1.91477   -0.159747

[:, :, 1, 5] =
  0.554726  -0.8693

In [5]:
# qwe == qwe2

true

In [6]:
# I change this function in order to ignore the indexes with labels -1 in nll function. We don't need this part anymore.
#=
function findindices(y,a::AbstractArray{<:Integer}; dims=1)
    n = length(a[a.!=-1]) # ignore indexes with -1
    indices = Vector{Int}(undef,n)
    if dims == 1                   # instances in first dimension
        y1 = size(y,1)
        y2 = div(length(y),y1)
        #if n != y2; throw(DimensionMismatch()); end
        @inbounds for j=1:n
            if a[j] != -1
                indices[j] = (j-1)*y1 + a[j]
            end
        end
    elseif dims == 2               # instances in last dimension
        y2 = size(y,ndims(y))
        y1 = div(length(y),y2)
        #if n != y1; throw(DimensionMismatch()); end
        @inbounds for j=1:n
            if a[j] != -1
                indices[j] = (a[j]-1)*y1 + j
            end
        end
    else
        error("findindices only supports dims = 1 or 2")
    end
    return indices
end
=#